## Load dataset

In [ ]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

common_voice["train"] = load_dataset("mozilla-foundation/common_voice_11_0", "zh-CN", data_dir = dataset_dir, split="train+validation", use_auth_token=True)
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "zh-CN", data_dir = dataset_dir, split="test", use_auth_token=True)

print(common_voice)

In [ ]:
# Remove irrelevant columns in the data
common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])

print(common_voice)

In [ ]:
# Save dataset to google drive
import os

dataset_dir = "G:\我的云端硬盘\whisper\ASR_data"
os.makedirs(dataset_dir, exist_ok = True)

common_voice.save_to_disk(dataset_dir)

In [ ]:
# Retrieve common voice from google drive
from datasets import load_dataset, DatasetDict

dataset_dir = 'G:\我的云端硬盘\whisper\ASR_data'

common_voice = DatasetDict.load_from_disk(dataset_dir)

print(common_voice)

## Prepare Feature Extractor, Tokenizer and Data

In [ ]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="zh-CN", task="transcribe")

In [ ]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

In [ ]:
# Create a whisper processor
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="zh-CN", task="transcribe")

In [ ]:
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

In [ ]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

In [ ]:
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=2)